# Get the Data
https://datos.gob.mx/busca/dataset/informacion-referente-a-casos-covid-19-en-mexico

http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip

In [32]:
# imports
import requests 
import pathlib
import os
from datetime import date
import zipfile

today = date.today()
currentDir = pathlib.Path().absolute()
dataDir = os.path.join(currentDir, "data")



covidDataUrl = "http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"
datasetZipTodayPath = "{dataDir}\{date}.zip".format(dataDir=dataDir, date=today)


print("Data Dir: {}".format(dataDir))
print("Dataset .zip path: {}".format(datasetZipTodayPath))

Data Dir: D:\OneDrive - bwedu\covid19mexico\data
Dataset .zip path: D:\OneDrive - bwedu\covid19mexico\data\2020-06-14.zip


### Functions

In [31]:
def download_url(url, save_path, chunk_size=128):
    r = requests.get(url, stream=True)
    open(save_path, 'wb').write(r.content)
    
    with zipfile.ZipFile(save_path, 'r') as zip_ref:
        zip_ref.extractall(dataDir)
    
    #with open(save_path, 'wb') as fd:
    #    for chunk in r.iter_content(chunk_size=chunk_size):
    #        fd.write(chunk)

In [33]:
if not os.path.isfile(datasetZipTodayPath):
    print("Downloading new covid-19 data...")
    download_url(url=covidDataUrl, save_path=datasetZipTodayPath)